#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### Chapter 16
**CH16A Predicting apartment prices with random forest**

using the airbnb dataset

version 0.92 2021-07-05


### NOTE: THIS IS NEW MATERIAL, NOT YET IN THE TEXTBOOK

In [ ]:
import math
import os
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from patsy import dmatrices
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.inspection import partial_dependence, permutation_importance
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier


## PART I
### Loading and preparing data 
----------------------------------------------

In [9]:
# !!! make sure you have run ch16-airbnb-prepare-london.ipynb before
area = "london"
data = pd.read_csv("/workspaces/codespaces-jupyter/data/airbnb_hackney_workfile_adj.csv")
data = data.loc[data.price.notna(), :]

In [10]:
def count_missing_values(df):
    return df.isna().sum()[df.isna().sum() > 0]

In [11]:
count_missing_values(data)

usd_cleaning_fee           1727
p_host_response_rate       1526
n_bathrooms                   9
n_review_scores_rating     1423
n_reviews_per_month        1349
n_beds                       12
n_days_since               1349
ln_beds                      12
f_bathroom                    9
f_number_of_reviews           1
ln_days_since              1350
ln_days_since2             1350
ln_days_since3             1350
n_days_since2              1349
n_days_since3              1349
ln_review_scores_rating    1423
f_minimum_nights              1
dtype: int64

In [ ]:
# Sample definition and preparation ---------------------------------------
# We focus on normal apartments, n<8
data = data.loc[lambda x: x["n_accommodates"] < 8]

In [ ]:
# copy a variable - purpose later, see at variable importance
data = data.assign(n_accommodates_copy=data.n_accommodates)

In [ ]:
# basic descr stat -------------------------------------------
data.describe()

In [ ]:
data.price.describe()

In [ ]:
data.price.describe()

In [ ]:
data.f_room_type.value_counts()

In [ ]:
data.f_property_type.value_counts()

In [ ]:
data.f_number_of_reviews.value_counts()

In [ ]:
# create train and holdout samples -------------------------------------------
# train is where we do it all, incl CV

# First pick a smaller than usual training set so that models run faster and check if works
# If works, start anew without these two lines

In [ ]:
data_train, data_holdout = train_test_split(data, train_size=0.7, random_state=42)

In [ ]:
data_train.shape, data_holdout.shape

In [ ]:
# Basic Variables inc neighnourhood
basic_vars = [
    "n_accommodates",
    "n_beds",
    "n_days_since",
    "f_property_type",
    "f_room_type",
    "f_bathroom",
    "f_cancellation_policy",
    "f_bed_type",
    "f_neighbourhood_cleansed",
]

# reviews
reviews = [
    "n_number_of_reviews",
    "flag_n_number_of_reviews",
    "n_review_scores_rating",
    "flag_review_scores_rating",
]

# Dummy variables
amenities = [col for col in data if col.startswith("d_")]

# interactions for the LASSO
# from ch14
X1 = [
    "n_accommodates:f_property_type",
    "f_room_type:f_property_type",
    "f_room_type:d_familykidfriendly",
    "d_airconditioning:f_property_type",
    "d_cats:f_property_type",
    "d_dogs:f_property_type",
]
# with boroughs
X2 = [
    "f_property_type:f_neighbourhood_cleansed",
    "f_room_type:f_neighbourhood_cleansed",
    "n_accommodates:f_neighbourhood_cleansed",
]

In [ ]:
predictors_1 = basic_vars
predictors_2 = basic_vars + reviews + amenities
predictors_E = basic_vars + reviews + amenities + X1 + X2

In [ ]:
categorical_columns = [col for col in predictors_2 if col.startswith("f_")]
numerical_columns = [col for col in predictors_2 if col not in categorical_columns]

Create sklearn preprocessor for encoding categorical variables to dummies

In [ ]:
preprocessor = ColumnTransformer(
    [
        ("cat", OneHotEncoder(), categorical_columns),
        ("num", "passthrough", numerical_columns),
    ]
)

In [ ]:
X = preprocessor.fit_transform(data_train[predictors_2])
X = pd.DataFrame(X, columns=preprocessor.get_feature_names_out())
y = data_train["price"]

## PART II
### RANDOM FORESTS 
-------------------------------------------------------

**Note:** n_estimators=500 in the R code.

Here, we set it to 30 because the model runs mutch faster, and this does not change the results substantively here – however in other cases might.

In [ ]:
# NOTE: set number of cores you want to run models
ncores = 7

In [ ]:
rfr = RandomForestRegressor(
    random_state=42,
    criterion="squared_error",
    n_estimators=30,
    oob_score=True,
    n_jobs=ncores,
)

tune_grid = {"max_features": [5, 7, 9], "min_samples_split": [6, 11]}

rf_random = GridSearchCV(
    rfr, tune_grid, cv=5, scoring="neg_root_mean_squared_error", verbose=3
)

rf_model_1 = rf_random.fit(X, y)

In [ ]:
rfr = RandomForestRegressor(
    random_state=42,
    criterion="squared_error",
    n_estimators=30,
    oob_score=True,
    n_jobs=ncores,
)

tune_grid = {
    "max_features": [8, 10, 12],
    "min_samples_split": [6, 11, 16],
}

rf_random = GridSearchCV(
    rfr,
    tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)

rf_model_2 = rf_random.fit(X, y)

### Table 16.1 Random forest RMSE by tuning parameters

In [ ]:
(
    pd.DataFrame(rf_model_2.cv_results_)[
        ["param_max_features", "param_min_samples_split", "mean_test_score"]
    ]
    .assign(
        mean_test_score=lambda x: x["mean_test_score"] * -1,
        Variables=lambda x: x["param_max_features"],
        Min_nodes=lambda x: x["param_min_samples_split"] - 1,
    )
    .pivot(index="Min_nodes", columns="Variables", values="mean_test_score")
    .round(2)
)

In [ ]:
pd.DataFrame(
    {
        "Min vars": [
            rf_model_1.best_estimator_.max_features,
            rf_model_2.best_estimator_.max_features,
        ],
        "Min nodes": [
            rf_model_1.best_estimator_.min_samples_split - 1,
            rf_model_2.best_estimator_.min_samples_split - 1,
        ],
    },
    ["Model A", "Model B"],
)

In [ ]:
rf_model_1_rmse = rf_model_1.cv_results_["mean_test_score"].max() * -1
rf_model_2_rmse = rf_model_2.cv_results_["mean_test_score"].max() * -1

pd.DataFrame(
    {"RMSE": [rf_model_1_rmse, rf_model_2_rmse]}, ["Model A", "Model B"]
).round(2)

## PART III
### MODEL DIAGNOSTICS 
---

In [ ]:
rf_model_2_var_imp_df = (
    pd.DataFrame(
        [rf_model_2.best_estimator_.feature_importances_, X.columns],
        index=["imp", "varname"],
    )
    .T.assign(
        imp_percentage=lambda x: x["imp"] / x["imp"].sum(),
        varname=lambda x: x["varname"]
        .str.replace("cat__f_room_type_", "Room type:", regex=False)
        .str.replace("cat__f_neighbourhood_cleansed_", "Borough:", regex=False)
        .str.replace("cat__f_cancellation_policy_", "Cancelation policy:", regex=False)
        .str.replace("cat__f_bed_type_", "Bed type:", regex=False)
        .str.replace("cat_f_property_type_", "Property type:", regex=False)
        .str.replace("num__", "", regex=False)
        .str.replace("cat__f_bathroom_0", "No bathroom")
        .str.replace("cat__f_bathroom_1", "One bathroom")
        .str.replace("cat__f_bathroom_2", "More than 1 bathroom"),
    )
    .sort_values(by=["imp"], ascending=False)
)

In [ ]:
def da_variable_importance_plot(
    data: pd.DataFrame,
    x: str = "imp_percentage",
    y: str = "varname",
    title: str | None = None,
):
    data = data.sort_values(by=x, ascending=False)

    sns.scatterplot(data=data, x=x, y=y, s=60)

    # Add horizontal lines from 0 to imp_percentage
    for _, row in data.iterrows():
        plt.hlines(y=row[y], xmin=0, xmax=row[x], linewidth=2.5)

    plt.xticks(np.arange(0, data[x].max(), 0.05))
    plt.gca().xaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))
    plt.title(title)
    plt.xlabel("Importance (Percent)")
    plt.ylabel("Variable Name")
    plt.show()

**1) full varimp plot, above a cutoff**

In [ ]:
cutoff = 0.013

plot_df = rf_model_2_var_imp_df.loc[lambda x: x.imp > cutoff]
da_variable_importance_plot(plot_df)

**2) full varimp plot, top 10 only**

In [ ]:
plot_df = rf_model_2_var_imp_df.head(10)
da_variable_importance_plot(plot_df)

#### 3) grouped variable importance - keep binaries created off factors together

First we do this by summing up the individual importances of factors - this is not correct, but it's in the first edition.

Simply summing up the individual importances of each dummy could underestimate the importance of the qualitative variable. This occurs because omitting a single category might not significantly impact performance as the remaining correlated categories can still provide the model with the necessary information. 

To address this issue, we need to assess the impact of including the entire categorical variable, not just the individual dummies. We create a pipeline which first encodes the categorical variables into dummy variables, then trains the model on the encoded data. Finally, we can employ a model-agnostic feature importance technique on this entire pipeline to estimate the contribution of each variable. This way, one categorical variableis either included or not during the calculation of variable importances.
 
Model-agnostic feature importance techniques are methods used to determine the importance of features in a model, regardless of the model type. These techniques work by assessing the impact of each feature on the model's predictions without relying on the internal workings of the model. One such technique is permutation feature importance, which randomly shuffles the values of variables and measures how much the fit of the prediction is decreased.

In [ ]:
rf_model_2_var_imp_df_grouped = (
    pd.DataFrame(
        [rf_model_2.best_estimator_.feature_importances_, X.columns],
        index=["imp", "varname"],
    )
    .T.assign(
        varname=lambda x: np.where(
            x["varname"].str.contains("cat__"),
            x["varname"].str.split("_").str[:-1].str.join("_"),
            x["varname"],
        )
    )
    .assign(
        varname=lambda x: x["varname"].str.replace("cat__", "").str.replace("num__", "")
    )
    .groupby("varname")[["imp"]]
    .sum()
    .reset_index()
    .assign(imp_percentage=lambda x: x["imp"] / x["imp"].sum())
    .sort_values(by=["imp"], ascending=False)
)

In [ ]:
plot_df = rf_model_2_var_imp_df_grouped.sort_values(by=["imp"], ascending=False).head(
    10
)
da_variable_importance_plot(
    plot_df, title="Top 10 most important variable - factor variables summed"
)

**OneHotEncoding and training the RandomForest model in a pipeline and calculating permutation importance - second edition**

In [ ]:
rf_best_pipeline = Pipeline(
    [
        ("preprocess", preprocessor),
        ("regressor", rf_model_2.best_estimator_),  # put best model to pipeline
    ]
)

In [ ]:
rf_best_pipeline.fit(data_train[predictors_2], data_train.price)

In [ ]:
# This takes a while
result = permutation_importance(
    rf_best_pipeline,
    data_train[predictors_2],
    data_train.price,
    n_repeats=10,
    random_state=45,
    n_jobs=-1,
)

In [ ]:
grouped_imp = pd.DataFrame(
    [result.importances_mean, data_train[predictors_2].columns],
    index=["imp", "varname"],
).T.assign(imp_percentage=lambda x: x["imp"] / x["imp"].sum())

In [ ]:
plot_df = grouped_imp.sort_values(by=["imp"], ascending=False).head(10)
da_variable_importance_plot(
    plot_df,
    title="Top 10 most important variable calculated with permutation importance",
)

## Partial Dependence Plots 
-------------------------------------------------------


In [ ]:
def set_ylim_for_pdp(pdp_results: pd.DataFrame):
    ymax = math.ceil(pdp_results["average"].max() / 10) * 10
    ymin = math.floor(pdp_results["average"].min() / 10) * 10
    plt.ylim(ymin, ymax)
    plt.yticks(np.arange(ymin, ymax + 1, 10))


def da_plot_partial_dependence(model, data: pd.DataFrame, variable: str, varlabel: str):

    pdp_results = partial_dependence(model, data, [variable], kind="average")

    pdp_results = pd.DataFrame(
        [pdp_results["average"][0], pdp_results["values"][0]],
        index=["average", "values"],
    ).T

    if pdp_results["values"].dtype == "object":
        linestyle = "none"
    else:
        linestyle = "-"

    sns.pointplot(
        data=pdp_results, x="values", y="average", scale=0.8, linestyle=linestyle
    )
    set_ylim_for_pdp(pdp_results)
    plt.grid(axis="x", linestyle="-", alpha=0.7)
    plt.xlabel(varlabel)
    plt.ylabel("Predicted price")
    plt.show()

In [ ]:
da_plot_partial_dependence(
    rf_best_pipeline,
    data_holdout[predictors_2],
    "n_accommodates",
    "Accomodates (person)",
)

In [ ]:
da_plot_partial_dependence(
    rf_best_pipeline, data_holdout[predictors_2], "f_room_type", "Room type"
)

### Subsample performance: RMSE / mean(y) 
---------------------------------------
NOTE  we do this on the holdout set.


In [ ]:
data_holdout_w_prediction = data_holdout.assign(
    predicted_price=rf_best_pipeline.predict(data_holdout[predictors_2])
)

create nice summary table of heterogeneity

In [ ]:
def calculate_rmse(groupby_obj):
    return (
        groupby_obj.apply(
            lambda x: root_mean_squared_error(x.predicted_price, x.price),
        )
        .to_frame(name="rmse")
        .assign(mean_price=groupby_obj.apply(lambda x: np.mean(x.price)).values)
        .assign(rmse_norm=lambda x: x.rmse / x.mean_price)
        .round(2)
    )

In [ ]:
# cheaper or more expensive flats - not used in book
grouped_object = data_holdout_w_prediction.assign(
    is_low_size=lambda x: np.where(x.n_accommodates <= 3, "small apt", "large apt")
).groupby("is_low_size")
accom_subset = calculate_rmse(grouped_object)

In [ ]:
grouped_object = data_holdout_w_prediction.loc[
    lambda x: x.f_neighbourhood_cleansed.isin(
        [
            "Westminster",
            "Camden",
            "Kensington and Chelsea",
            "Tower Hamlets",
            "Hackney",
            "Newham",
        ]
    )
].groupby("f_neighbourhood_cleansed")
neightbourhood_subset = calculate_rmse(grouped_object)

In [ ]:
grouped_object = data_holdout_w_prediction.loc[
    lambda x: x.f_property_type.isin(["Apartment", "House"])
].groupby("f_property_type")
proptype_subset = calculate_rmse(grouped_object)

In [ ]:
all_holdout = (
    pd.DataFrame(
        [
            root_mean_squared_error(
                data_holdout_w_prediction.price,
                data_holdout_w_prediction.predicted_price,
            ),
            data_holdout_w_prediction.price.mean(),
        ],
        index=["rmse", "mean_price"],
    )
    .T.assign(rmse_norm=lambda x: x.rmse / x.mean_price)
    .round(2)
)
all_holdout.index = ["All"]

In [ ]:
type_rows = pd.DataFrame(
    None,
    index=["Apartment size", "Type", "Borough"],
    columns=["rmse", "mean_price", "rmse_norm"],
).fillna("")

### Table 16.2 Performance across subsamples

In [ ]:
pd.concat(
    [
        type_rows.iloc[[0]],
        accom_subset,
        type_rows.iloc[[1]],
        proptype_subset,
        type_rows.iloc[[2]],
        neightbourhood_subset,
        all_holdout,
    ]
)

## SHAP

In [ ]:
import shap

In [ ]:
X_holdout = preprocessor.fit_transform(data_holdout[predictors_2])
X_holdout = pd.DataFrame(X_holdout, columns=preprocessor.get_feature_names_out())

Calculate SHAP values for our best model

**NOTE:** Again, we do this on the holdout set!

In [ ]:
explainer = shap.Explainer(rf_best_pipeline["regressor"].predict, X_holdout)
shap_values = explainer(X_holdout)

### Beeswarm plot of SHAP values

The beeswarm plot is designed to display an information-dense summary of how the top features in a dataset impact the model’s output. Each instance the given explanation is represented by a single dot on each feature row. The x position of the dot is determined by the SHAP value of that feature, and dots “pile up” along each feature row to show density. Color is used to display the original value of a feature. In the plot below we can see that Entire home/apt is the most important feature on average, and than Entire home/apt-s are more expensive.



In [ ]:
shap.plots.beeswarm(
    shap_values, max_display=15, color=plt.get_cmap("viridis_r"), show=True
)

Can do the same with **SHAP values scaled log**. This might come handy, when the distribution of SHAP values are skewed

In [ ]:
shap.plots.beeswarm(
    shap_values, max_display=15, log_scale=True, color=plt.get_cmap("viridis_r")
)

You can also display the **SHAP values in absolute**, on a beeswarm plot

In [ ]:
shap.plots.beeswarm(shap_values.abs, color=color[0])

Or on a barplot

In [ ]:
shap.plots.bar(shap_values)

### Explanining predictions for a unit of observation (airbnb)

Let's look at SHAP values for the third observation in the holdout set. 

- .values array contains the shap values
- .base_values contains the expected value (intercept/constant in OLS terms)
- .data contains the feature values for the observation

In [ ]:
shap_values[2]

#### Waterfall plot
The waterfall plot shows how the sum of all the SHAP values equals the difference between the prediction $f(x)$ and the expected value $E[f(x)]$. Waterfall plots are designed to display explanations for individual predictions, so they expect a single row of an Explanation object as input. The bottom of a waterfall plot starts as the expected value of the model output, and then each row shows how the positive (red) or negative (blue) contribution of each feature moves the value from the expected model output over the background dataset to the model output for this prediction.

In [ ]:
shap.plots.waterfall(shap_values[2])

Same plot, but SHAP values for another observation below.

Note, that for a same predictor (eg. f_room_type_Private room = 0) the SHAP values are different for the airbnb above (+5.29) and above (+6.88).

In [ ]:
shap.plots.waterfall(shap_values[3])

## Lime


In [ ]:
feature_names = categorical_columns + numerical_columns
data_train_lime = data_train[feature_names].copy()
data_holdout_lime = data_holdout[feature_names].copy()

continous_and_ordinal_vars = [col for col in feature_names if col.startswith("n_")]
categorical_vars = [col for col in feature_names if not col.startswith("n_")]

In [ ]:
encoder = ColumnTransformer(
    [
        ("c", OrdinalEncoder(), categorical_vars),
        ("n", "passthrough", continous_and_ordinal_vars),
    ]
).fit(data_train_lime)

columns = [
    col.replace("n__", "").replace("c__", "") for col in encoder.get_feature_names_out()
]
X_train_lime = pd.DataFrame(encoder.transform(data_train_lime), columns=columns)
X_holdout_lime = pd.DataFrame(encoder.transform(data_holdout_lime), columns=columns)


numerical_indices = [
    X_train_lime.columns.get_loc(col) for col in continous_and_ordinal_vars
]
categorical_indices = [X_train_lime.columns.get_loc(col) for col in categorical_vars]

categorical_names = {
    idx: cat
    for idx, cat in zip(categorical_indices, encoder.transformers_[0][1].categories_)
}

In [ ]:
preprocessing = ColumnTransformer(
    [
        ("cat", OneHotEncoder(), categorical_indices),
        ("num", "passthrough", numerical_indices),
    ]
)


rf_best_pipeline = Pipeline(
    [
        ("preprocess", preprocessing),
        ("regressor", rf_model_2.best_estimator_),  # best model
    ]
)

In [ ]:
rf_best_pipeline.fit(X_train_lime, data_train.price)

In [ ]:
predict_fn = lambda x: rf_best_pipeline.predict(x).astype(float)

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(
    X_holdout_lime.to_numpy(),
    feature_names=X_holdout_lime.columns.tolist(),
    categorical_features=categorical_indices,
    categorical_names=categorical_names,
    mode="regression",
    random_state=1237,
)

In [ ]:
instance = X_holdout_lime.iloc[[2]]

In [ ]:
from sklearn.linear_model import Ridge

exp = explainer.explain_instance(
    instance.to_numpy()[0],
    predict_fn,
    num_features=20,
    num_samples=1000,
    distance_metric="euclidean",
    model_regressor=Ridge(
        alpha=1, fit_intercept=True, random_state=explainer.random_state
    ),
)

exp.as_pyplot_figure()
plt.show()

In [ ]:
exp.predicted_value

In [ ]:
exp.local_pred[0]

In [ ]:
exp.intercept[0]

In [ ]:
sum([i[1] for i in exp.as_list()])

## Interactions

Look at interactions from `ch14-airbnb-reg`

In [ ]:
(
    "f_room_type*d_familykidfriendly",
    "f_room_type*f_property_type",
    "f_property_type*d_airconditioning",
    "f_property_type*d_cats",
    "f_property_type*d_dogs",
)

To help reveal these interactions we can color by another feature. If we pass the whole explanation tensor to the color argument **the scatter plot will pick the best feature to color by.**

Get best proposed interaction by SHAP values for room types. Note that the grey area on the plot corresponds to the distribution of the feature in the data.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4), sharey=True)
fig.suptitle(
    "SHAP values and best proposed interaction for room types",
    fontsize=15,
)
shap.plots.scatter(
    shap_values[:, "f_room_type_Entire home/apt"],
    color=shap_values,
    show=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax1,
)
plt.ylabel("SHAP values")
plt.xlabel("Entire home/apt")
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
shap.plots.scatter(
    shap_values[:, "f_room_type_Private room"],
    color=shap_values,
    show=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax2,
)
plt.ylabel(None)
plt.xlabel("Private room")
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
shap.plots.scatter(
    shap_values[:, "f_room_type_Shared room"],
    color=shap_values,
    show=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax3,
)
plt.ylabel(None)
plt.xlabel("Shared room")
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
plt.show()

It turned out, that for the `f_room_type` variable the best interaction (at least based on RF and SHAP) would be the `f_bathroom` which we did not choose in ch14.

Let's check for `d_airconditioning`, `d_dogs` and `d_cats`.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4), sharey=True)
fig.suptitle(
    "SHAP values and best proposed interaction for d_airconditioning, d_cats and d_dogs",
    fontsize=15,
)
shap.plots.scatter(
    shap_values[:, "d_airconditioning"],
    color=shap_values,
    show=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax1,
)
plt.ylabel("SHAP values")
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
shap.plots.scatter(
    shap_values[:, "d_cats"],
    color=shap_values,
    show=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax2,
)
plt.ylabel(None)
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
shap.plots.scatter(
    shap_values[:, "d_dogs"],
    color=shap_values,
    show=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax3,
)
plt.ylabel(None)
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
plt.show()

The distribution in grey is switched off

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4), sharey=True)
fig.suptitle(
    "SHAP values and best proposed interaction for d_airconditioning, d_cats and d_dogs",
    fontsize=15,
)
shap.plots.scatter(
    shap_values[:, "d_airconditioning"],
    color=shap_values,
    show=False,
    hist=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax1,
)
plt.ylabel("SHAP values")
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
shap.plots.scatter(
    shap_values[:, "d_cats"],
    color=shap_values,
    show=False,
    hist=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax2,
)
plt.ylabel(None)
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
shap.plots.scatter(
    shap_values[:, "d_dogs"],
    color=shap_values,
    show=False,
    hist=False,
    cmap=plt.get_cmap("viridis_r"),
    ax=ax3,
)
plt.ylabel(None)
plt.xlim(-0.2, 1.2)
plt.xticks((0, 1), labels=(0, 1))
plt.show()

Take a look at a discrete feature, `n_accommodates`

In [ ]:
shap.plots.scatter(
    shap_values[:, "n_accommodates"], color=shap_values, cmap=plt.get_cmap("viridis_r")
)

## PART IV
### HORSERACE: compare with other models 
-----------------------------------------------

1. OLS with dummies for area

 using model B

In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

ols_model = LinearRegression().fit(X, y)

# y_test, X_test = dmatrices("price ~ " + " + ".join(predictors_2), data_holdout)

y_hat = ols_model.predict(X)

ols_rmse = root_mean_squared_error(y, y_hat)
ols_rmse

In [ ]:
ols_model_coeffs_df = pd.DataFrame(
    ols_model.coef_.tolist()[0],
    index=X.design_info.column_names,
    columns=["ols_coefficient"],
).assign(ols_coefficient=lambda x: x.ols_coefficient.round(3))

In [ ]:
ols_model_coeffs_df

2.  LASSO

using extended model w interactions

The parameter l1_ratio corresponds to alpha in the glmnet R package while alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio = 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable, unless you supply your own sequence of alpha.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [ ]:
lasso_model = ElasticNet(l1_ratio=1, normalize=True, fit_intercept=True)

In [ ]:
lasso_model_cv = GridSearchCV(
    lasso_model,
    {"alpha": [i / 100 for i in range(1, 26, 1)]},
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)

In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_E), data_train)

In [ ]:
lasso_model_cv.fit(X, y.ravel())

In [ ]:
pd.DataFrame(
    lasso_model_cv.best_estimator_.coef_.tolist(),
    index=X.design_info.column_names,
    columns=["lasso_coefficient"],
).assign(lasso_coefficient=lambda x: x.lasso_coefficient.round(3)).loc[
    lambda x: x.lasso_coefficient != 0
]

In [ ]:
lasso_rmse = (
    pd.DataFrame(lasso_model_cv.cv_results_)
    .loc[lambda x: x.param_alpha == lasso_model_cv.best_estimator_.alpha]
    .mean_test_score.values[0]
    * -1
)
lasso_rmse

3. CART model

In [ ]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

In [ ]:
cart_model = DecisionTreeClassifier(random_state=2018, criterion="gini")

In [ ]:
# Get potential ccp_alpha parameters

path = cart_model.cost_complexity_pruning_path(X, y.ravel())
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
# apply random search to select a "best" alpha
# RandomizedSearchCV does not calculate all potential alphas, just a random subset

cart_model_cv = RandomizedSearchCV(
    cart_model,
    {"ccp_alpha": ccp_alphas},
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)


cart_model_cv.fit(X, y.ravel())

In [ ]:
cart_rmse = (
    pd.DataFrame(cart_model_cv.cv_results_)
    .loc[lambda x: x.param_ccp_alpha == cart_model_cv.best_estimator_.ccp_alpha]
    .mean_test_score.values[0]
    * -1
)
cart_rmse

4. GBM

**NOTE:** These models run for a **very long time** -- needs further investigations.

In [ ]:
gbm = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=20)

tune_grid = {"n_estimators": [i for i in range(200, 500, 50)], "max_depth": [1, 5, 10]}

gbm_model_cv = GridSearchCV(
    gbm, tune_grid, cv=5, scoring="neg_root_mean_squared_error", verbose=10, n_jobs=-1
)

In [ ]:
categorical_columns = [col for col in predictors_2 if col.startswith("f_")]
numerical_columns = [col for col in predictors_2 if col not in categorical_columns]

categorical_encoder = OneHotEncoder(handle_unknown="ignore")

preprocessing = ColumnTransformer(
    [
        ("cat", categorical_encoder, categorical_columns),
        ("num", "passthrough", numerical_columns),
    ]
)

gbm_pipe = Pipeline(
    [("preprocess", preprocessing), ("regressor", gbm_model_cv)], verbose=True
)

In [ ]:
gbm_pipe.fit(data_train[predictors_2], data_train.price)

In [ ]:
gbm_rmse = gbm_pipe.steps[-1][1].best_score_ * -1

the next will be in final model, loads of tuning

In [ ]:
gbm_broad = GradientBoostingRegressor()

In [ ]:
tune_grid = {
    "n_estimators": [i for i in range(50, 500, 50)],
    "max_depth": [1, 5, 10],
    "learning_rate": [0.02, 0.05, 0.1, 0.15, 0.2],
    "min_samples_split": [5, 10, 20, 30],
}

gbm_model_cv_broad = GridSearchCV(
    gbm_broad,
    tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=10,
)

In [ ]:
categorical_columns = [col for col in predictors_2 if col.startswith("f_")]
numerical_columns = [col for col in predictors_2 if col not in categorical_columns]

categorical_encoder = OneHotEncoder(handle_unknown="ignore")

preprocessing = ColumnTransformer(
    [
        ("cat", categorical_encoder, categorical_columns),
        ("num", "passthrough", numerical_columns),
    ]
)

gbm_pipe_broad = Pipeline(
    [("preprocess", preprocessing), ("regressor", gbm_model_cv_broad)], verbose=True
)

In [ ]:
gbm_pipe_broad.fit(data_train[predictors_2], data_train.price)

In [ ]:
gbm_broad_rmse = gbm_pipe_broad.steps[-1][1].best_score_ * -1

### Table 16.3 Predictive performance of different models

In [ ]:
pd.DataFrame(
    {
        "Model": [
            "Linear regression (OLS)",
            "Linear regression (LASSO)",
            "Regression Tree (CART)",
            "Random forest (basic tuning)",
            "Random forest (autotuned)",
            "GBM (basic tuning)",
            "GBM (broad tuning)",
        ],
        "RMSE": [
            ols_rmse,
            lasso_rmse,
            cart_rmse,
            rf_model_1_rmse,
            rf_model_2_rmse,
            gbm_rmse,
            gbm_broad_rmse,
        ],
    }
).round(1)